In [ ]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import time
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [ ]:
iotd = pd.read_csv("iot23_not_preprocessed.csv")

In [ ]:
cic = pd.read_csv("cicids2017_preprocessed.csv")

In [ ]:
#5:5 비율로 학습하기 위해 IoT-23 개수만큼 CICIDS2017을 랜덤 추출
cicd = cic.sample(n=1444674, replace=False, axis=0)

In [ ]:
del iotd['Unnamed: 0']

In [ ]:
iotd['duration'] = iotd['duration'].str.replace('-','0')
iotd['orig_bytes'] = iotd['orig_bytes'].str.replace('-','0')
iotd['resp_bytes'] = iotd['resp_bytes'].str.replace('-','0')
iotd.loc[(iotd.label == '-   Malicious   PartOfAHorizontalPortScan'), 'label'] = 'PartOfAHorizontalPortScan'
iotd.loc[(iotd.label == '(empty)   Malicious   PartOfAHorizontalPortScan'), 'label'] = 'PartOfAHorizontalPortScan'
iotd.loc[(iotd.label == '-   Malicious   Okiru'), 'label'] = 'Okiru'
iotd.loc[(iotd.label == '(empty)   Malicious   Okiru'), 'label'] = 'Okiru'
iotd.loc[(iotd.label == '-   Benign   -'), 'label'] = 'Benign'
iotd.loc[(iotd.label == '(empty)   Benign   -'), 'label'] = 'Benign'
iotd.loc[(iotd.label == '-   Malicious   DDoS'), 'label'] = 'DDoS'
iotd.loc[(iotd.label == '-   Malicious   C&C'), 'label'] = 'C&C'
iotd.loc[(iotd.label == '(empty)   Malicious   C&C'), 'label'] = 'C&C'
iotd.loc[(iotd.label == '-   Malicious   Attack'), 'label'] = 'Attack'
iotd.loc[(iotd.label == '(empty)   Malicious   Attack'), 'label'] = 'Attack'
iotd.loc[(iotd.label == '-   Malicious   C&C-HeartBeat'), 'label'] = 'C&C-HeartBeat'
iotd.loc[(iotd.label == '(empty)   Malicious   C&C-HeartBeat'), 'label'] = 'C&C-HeartBeat'
iotd.loc[(iotd.label == '-   Malicious   C&C-FileDownload'), 'label'] = 'C&C-FileDownload'
iotd.loc[(iotd.label == '-   Malicious   C&C-Torii'), 'label'] = 'C&C-Torii'
iotd.loc[(iotd.label == '-   Malicious   C&C-HeartBeat-FileDownload'), 'label'] = 'C&C-HeartBeat-FileDownload'
iotd.loc[(iotd.label == '-   Malicious   FileDownload'), 'label'] = 'FileDownload'
iotd.loc[(iotd.label == '-   Malicious   C&C-Mirai'), 'label'] = 'C&C-Mirai'
iotd.loc[(iotd.label == '-   Malicious   Okiru-Attack'), 'label'] = 'Okiru-Attack'

In [ ]:
#cic에 없는 feature명 추가
cicd = cicd.reindex(columns = cicd.columns.tolist() +['service', 'conn_state', 'local_orig', 'local_resp', 'missed_bytes', 'history' ])

In [ ]:
#필요 없는 ip, port 관련 feature drop
cicd = cicd.drop(columns=['Flow ID', 'Source IP', 'Source Port', 'Destination IP', 'Destination Port','Timestamp'])

In [ ]:
iotd = iotd.drop(columns=['uid', 'id.orig_h', 'id.orig_p', 'id.resp_h', 'id.resp_p'])

In [ ]:
iotd = iotd.rename(columns={'ts':'Timestamp', 'duration':'Flow Duration','orig_pkts':'Total Fwd Packets', 'resp_pkts':'Total Backward Packets', 'proto':'Protocol', 'orig_bytes':'Fwd Packet Length Mean' , 'resp_bytes': 'Bwd Packet Length Mean' , 'orig_ip_bytes': 'Fwd Packets/s' ,'resp_ip_bytes': 'Bwd Packets/s' ,'label': 'Label' })

In [ ]:
iotd = iotd.replace('Benign', 'BENIGN' )
iotd = iotd.replace('C&C-HeartBeat', 'C&C' )
iotd = iotd.replace('C&C-FileDownload', 'C&C' )
iotd = iotd.replace('C&C-Torii', 'C&C' )
iotd = iotd.replace('C&C-HeartBeat-FileDownload', 'C&C' )
iotd = iotd.replace('C&C-Mirai', 'C&C' )
iotd = iotd.replace('PartOfAHorizontalPortScan', 'PortScan' )

iotd = iotd.replace('tcp', '6' )
iotd = iotd.replace('udp', '17' )
iotd = iotd.replace('icmp', '1' )

In [ ]:
cicd['Label'].value_counts()

In [ ]:
cicd = cicd.replace('DoS Hulk', 'DoS' )
cicd = cicd.replace('DoS GoldenEye', 'DoS' )
cicd = cicd.replace('DoS slowloris', 'DoS' )
cicd = cicd.replace('DoS Slowhttptest', 'DoS' )

In [ ]:
#iot23에 없는 피처명 추가
iotd = iotd.reindex(columns = iotd.columns.tolist() + ['Total Length of Fwd Packets', 'Total Length of Bwd Packets', 'Fwd Packet Length Max', 'Fwd Packet Length Min' , 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length', 'Bwd Header Length', 'Min Packet Length', 'Max Packet Length', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG Flag Count', 'CWE Flag Count', 'ECE Flag Count', 'Down/Up Ratio', 'Average Packet Size', 'Avg Fwd Segment Size', 'Avg Bwd Segment Size', 'Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk', 'Fwd Avg Bulk Rate', 'Bwd Avg Bytes/Bulk', 'Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate', 'Subflow Fwd Packets', 'Subflow Fwd Bytes', 'Subflow Bwd Packets', 'Subflow Bwd Bytes', 'Init_Win_bytes_forward', 'Init_Win_bytes_backward', 'act_data_pkt_fwd', 'min_seg_size_forward', 'Active Mean', 'Active Std', 'Active Max', 'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max', 'Idle Min', 'External IP'])

In [ ]:
cicd = cicd.drop(columns=['External IP'])

In [ ]:
#데이터 합치기
frames = [iotd, cicd]
df_merge = pd.concat(frames)

In [ ]:
#결측치 체크
cicd.isna().sum()

In [ ]:
#결측치 채우기
cicd.fillna(-1,inplace=True)
iotd.fillna(-1,inplace=True)
df_merge.fillna(-1,inplace=True)

In [ ]:
iotd = iotd.drop(columns=['uid', 'id.orig_h', 'id.resp_h', 'service', 'Flow Duration', 'Fwd Packet Length Mean', 'Bwd Packet Length Mean', 'local_orig', 'local_resp', 'history'])

In [ ]:
df_merge = df_merge.drop(columns=['Timestamp'])

In [ ]:
df_merge['Protocol'] = df_merge['Protocol'].replace('tcp',6)
df_merge['Protocol'] = df_merge['Protocol'].replace('udp',17)
df_merge['Protocol'] = df_merge['Protocol'].replace('icmp',1)

In [ ]:
df_merge = df_merge.astype({'Protocol':'float'})

In [ ]:
df_merge['conn_state'] = df_merge['conn_state'].replace('-1',0)
df_merge['conn_state'] = df_merge['conn_state'].replace('S0',1)
df_merge['conn_state'] = df_merge['conn_state'].replace('OTH',2)
df_merge['conn_state'] = df_merge['conn_state'].replace('SF',3)
df_merge['conn_state'] = df_merge['conn_state'].replace('REJ',4)
df_merge['conn_state'] = df_merge['conn_state'].replace('S3',5)
df_merge['conn_state'] = df_merge['conn_state'].replace('RSTR',6)
df_merge['conn_state'] = df_merge['conn_state'].replace('RSTO',7)
df_merge['conn_state'] = df_merge['conn_state'].replace('S1',8)
df_merge['conn_state'] = df_merge['conn_state'].replace('RSTOS0',9)
df_merge['conn_state'] = df_merge['conn_state'].replace('S2',10)
df_merge['conn_state'] = df_merge['conn_state'].replace('SH',11)
df_merge['conn_state'] = df_merge['conn_state'].replace('RSTRH',12)
df_merge['conn_state'] = df_merge['conn_state'].replace('SHR',13)

In [ ]:
iotd['conn_state'] = iotd['conn_state'].replace('-1',0)
iotd['conn_state'] = iotd['conn_state'].replace('S0',1)
iotd['conn_state'] = iotd['conn_state'].replace('OTH',2)
iotd['conn_state'] = iotd['conn_state'].replace('SF',3)
iotd['conn_state'] = iotd['conn_state'].replace('REJ',4)
iotd['conn_state'] = iotd['conn_state'].replace('S3',5)
iotd['conn_state'] = iotd['conn_state'].replace('RSTR',6)
iotd['conn_state'] = iotd['conn_state'].replace('RSTO',7)
iotd['conn_state'] = iotd['conn_state'].replace('S1',8)
iotd['conn_state'] = iotd['conn_state'].replace('RSTOS0',9)
iotd['conn_state'] = iotd['conn_state'].replace('S2',10)
iotd['conn_state'] = iotd['conn_state'].replace('SH',11)
iotd['conn_state'] = iotd['conn_state'].replace('RSTRH',12)
iotd['conn_state'] = iotd['conn_state'].replace('SHR',13)

In [ ]:
df_merge = df_merge.astype({'conn_state':'float'})

In [ ]:
df_merge = df_merge.drop(columns=['service', 'local_orig', 'local_resp', 'history'])

In [ ]:
X = df_merge[['Protocol', 'Flow Duration', 'Fwd Packet Length Mean', 'Bwd Packet Length Mean', 'conn_state', 'missed_bytes', 'Total Fwd Packets', 'Fwd Packets/s', 'Total Backward Packets', 'Bwd Packets/s','Total Length of Fwd Packets', 'Total Length of Bwd Packets', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length', 'Bwd Header Length', 'Min Packet Length', 'Max Packet Length', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG Flag Count', 'CWE Flag Count', 'ECE Flag Count', 'Down/Up Ratio', 'Average Packet Size', 'Avg Fwd Segment Size', 'Avg Bwd Segment Size', 'Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk', 'Fwd Avg Bulk Rate', 'Bwd Avg Bytes/Bulk', 'Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate', 'Subflow Fwd Packets', 'Subflow Fwd Bytes', 'Subflow Bwd Packets', 'Subflow Bwd Bytes', 'Init_Win_bytes_forward', 'Init_Win_bytes_backward', 'act_data_pkt_fwd', 'min_seg_size_forward', 'Active Mean', 'Active Std', 'Active Max', 'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max', 'Idle Min' ]]
y = df_merge['Label'] 

labels = X.columns

from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X, y)

importance = model.feature_importances_
std = np.std([tree.feature_importances_ for tree in model.estimators_], axis=0)
indices = np.argsort(importance)[::-1]

print("Feature ranking:")

for f in range(X.shape[1]):
    print("{}. {} ({:.5f})".format(f + 1, X.columns[indices][f], importance[indices[f]]))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier()

In [ ]:
#feature importance 실험 결과 높게 나온 feature 6개를 각각의 데이터셋에 추가(공통 feature를 제외한 순위)
iotd['Fwd IAT Min']=np.NaN
iotd['Packet Length Std']=np.NaN
iotd['Total Length of Fwd Packets']=np.NaN
iotd['Bwd IAT Mean']=np.NaN
iotd['Bwd Packet Length Min']=np.NaN
iotd['Total Length of Fwd Packets']=np.NaN

In [ ]:
cicd['conn_state']=np.NaN

In [ ]:
X = df_merge[['Protocol', 'Flow Duration', 'Bwd Packet Length Mean',  'Fwd Packet Length Mean',  'Fwd Packets/s', 'Bwd Packets/s','Total Fwd Packets', 'Total Backward Packets', 'Fwd IAT Min', 'conn_state', 'Packet Length Std', 'Total Length of Fwd Packets', 'Bwd IAT Mean', 'Bwd Packet Length Min']]
Y = df_merge['Label']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=10, test_size=0.2)

In [ ]:
Z = cicd[['Protocol', 'Flow Duration', 'Bwd Packet Length Mean',  'Fwd Packet Length Mean',  'Fwd Packets/s', 'Bwd Packets/s','Total Fwd Packets', 'Total Backward Packets', 'Fwd IAT Min', 'conn_state', 'Packet Length Std', 'Total Length of Fwd Packets', 'Bwd IAT Mean', 'Bwd Packet Length Min']]
W = cicd['Label']
Z_train, Z_test, W_train, W_test = train_test_split(Z, W, random_state=10, test_size=0.4)

In [ ]:
X_test = Z_test
Y_test = W_test

In [ ]:
start = time.time()
print('program start...')
print()

RF.fit(X_train, Y_train)  # fit() : 훈련 수행
print()
print('prediction:')
y_pred = RF.predict(X_test)  # predict() : 모델 훈련 후 예측
print(y_pred)
print()

print('Score:')
score = RF.score(X_test,Y_test)  # score() : 평가 점수 계산
print(score)

end = time.time()  # 현재 시각 구하기
print('program end...')
print()
print('time cost: ')
print(end - start, 'seconds')
print("Classifiction Report :")
print(classification_report(Y_test, y_pred))